In [ ]:
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import time
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse.linalg import svds
from sklearn.model_selection import train_test_split
from datetime import datetime

# --------- 1. UTILITY FUNCTIONS ---------

def load_processed_data(data_dir="processed_data"):
    """
    Load the preprocessed data created in Phase 1
    """
    print("Loading processed data...")
    
    # Load ratings data splits
    train = pd.read_csv(os.path.join(data_dir, 'train_ratings.csv'))
    val = pd.read_csv(os.path.join(data_dir, 'val_ratings.csv'))
    test = pd.read_csv(os.path.join(data_dir, 'test_ratings.csv'))
    
    # Load feature data
    movies = pd.read_csv(os.path.join(data_dir, 'movies_processed.csv'))
    user_features = pd.read_csv(os.path.join(data_dir, 'user_features.csv'))
    item_features = pd.read_csv(os.path.join(data_dir, 'item_features.csv'))
    
    # Load ID mappings
    user_map_df = pd.read_csv(os.path.join(data_dir, 'user_id_map.csv'))
    movie_map_df = pd.read_csv(os.path.join(data_dir, 'movie_id_map.csv'))
    
    # Convert mappings to dictionaries
    user_map = dict(zip(user_map_df['userId'], user_map_df['matrix_idx']))
    movie_map = dict(zip(movie_map_df['movieId'], movie_map_df['matrix_idx']))
    
    # Reverse mappings (matrix index to ID)
    user_map_rev = dict(zip(user_map_df['matrix_idx'], user_map_df['userId']))
    movie_map_rev = dict(zip(movie_map_df['matrix_idx'], movie_map_df['movieId']))
    
    # Load sparse matrices
    sparse_dir = os.path.join(data_dir, 'sparse_matrices')
    train_matrix = sparse.load_npz(os.path.join(sparse_dir, 'train_matrix.npz'))
    val_matrix = sparse.load_npz(os.path.join(sparse_dir, 'val_matrix.npz'))
    test_matrix = sparse.load_npz(os.path.join(sparse_dir, 'test_matrix.npz'))
    
    print("Data loading complete!")
    
    return {
        'train': train,
        'val': val,
        'test': test,
        'movies': movies,
        'user_features': user_features,
        'item_features': item_features,
        'user_map': user_map,
        'movie_map': movie_map,
        'user_map_rev': user_map_rev,
        'movie_map_rev': movie_map_rev,
        'train_matrix': train_matrix,
        'val_matrix': val_matrix,
        'test_matrix': test_matrix
    }

def evaluate_model(predictions, true_ratings, name="Model"):
    """
    Calculate evaluation metrics for a recommendation model
    
    Args:
        predictions: Predicted ratings
        true_ratings: Ground truth ratings
        name: Model name for display purposes
    
    Returns:
        Dictionary of metrics
    """
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(true_ratings, predictions))
    mae = np.mean(np.abs(true_ratings - predictions))
    
    # Calculate accuracy within different thresholds
    within_1 = np.mean(np.abs(true_ratings - predictions) <= 1.0)
    within_05 = np.mean(np.abs(true_ratings - predictions) <= 0.5)
    
    print(f"\n{name} Performance:")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"Within 0.5 stars: {within_05:.2%}")
    print(f"Within 1.0 stars: {within_1:.2%}")
    
    return {
        'model': name,
        'rmse': rmse,
        'mae': mae,
        'within_05': within_05,
        'within_1': within_1
    }

def get_user_rated_items(user_idx, train_matrix):
    """Get the items rated by a specific user in the training set"""
    user_ratings = train_matrix[user_idx].toarray().flatten()
    return np.where(user_ratings > 0)[0]

def get_top_n_recommendations(predictions, user_idx, n=10, exclude_rated=True, train_matrix=None):
    """
    Get top N movie recommendations for a user based on predictions
    
    Args:
        predictions: Matrix of predicted ratings
        user_idx: User index in the matrix
        n: Number of recommendations to return
        exclude_rated: Whether to exclude items the user has already rated
        train_matrix: Training matrix (needed if exclude_rated is True)
    
    Returns:
        List of top N movie indices
    """
    user_pred = predictions[user_idx].copy()
    
    if exclude_rated and train_matrix is not None:
        # Get items the user has already rated and set their predictions to -inf
        rated_items = get_user_rated_items(user_idx, train_matrix)
        user_pred[rated_items] = -np.inf
    
    # Get the indices of top N predictions
    top_indices = np.argsort(user_pred)[::-1][:n]
    
    return top_indices

# --------- 2. BASELINE MODELS ---------

class GlobalAverageModel:
    """Simple baseline that predicts the global average rating for all users and items"""
    
    def __init__(self):
        self.global_avg = None
    
    def fit(self, train_matrix):
        """Calculate the global average rating from training data"""
        start_time = time.time()
        # Calculate global average (ignoring zeros)
        ratings = train_matrix.data
        self.global_avg = ratings.mean()
        print(f"Global average rating: {self.global_avg:.4f}")
        print(f"Training time: {time.time() - start_time:.2f} seconds")
        return self
    
    def predict(self, user_indices, item_indices):
        """Predict using the global average for all user-item pairs"""
        return np.full(len(user_indices), self.global_avg)
    
    def predict_all(self):
        """Return the global average prediction for all possible user-item pairs"""
        return self.global_avg

class UserItemBiasModel:
    """Baseline model that incorporates user and item biases"""
    
    def __init__(self, reg_param=0.1):
        self.global_avg = None
        self.user_biases = None
        self.item_biases = None
        self.reg_param = reg_param  # Regularization parameter
    
    def fit(self, train_matrix):
        """Calculate global average, user biases, and item biases"""
        start_time = time.time()
        
        # Get the nonzero values in train_matrix
        ratings = train_matrix.data
        rows, cols = train_matrix.nonzero()
        n_users, n_items = train_matrix.shape
        
        # Calculate global average
        self.global_avg = ratings.mean()
        
        # Initialize bias arrays
        self.user_biases = np.zeros(n_users)
        self.item_biases = np.zeros(n_items)
        
        # Calculate user biases
        for u in range(n_users):
            user_ratings = train_matrix[u].data
            if len(user_ratings) > 0:
                self.user_biases[u] = user_ratings.mean() - self.global_avg
                # Apply regularization
                self.user_biases[u] *= len(user_ratings) / (len(user_ratings) + self.reg_param)
        
        # Calculate item biases
        for i in range(n_items):
            # Extract all ratings for item i (need to transpose matrix for efficiency)
            item_ratings = train_matrix.T[i].data
            if len(item_ratings) > 0:
                self.item_biases[i] = item_ratings.mean() - self.global_avg
                # Apply regularization
                self.item_biases[i] *= len(item_ratings) / (len(item_ratings) + self.reg_param)
        
        print(f"Global average: {self.global_avg:.4f}")
        print(f"User bias range: [{self.user_biases.min():.4f}, {self.user_biases.max():.4f}]")
        print(f"Item bias range: [{self.item_biases.min():.4f}, {self.item_biases.max():.4f}]")
        print(f"Training time: {time.time() - start_time:.2f} seconds")
        
        return self
    
    def predict(self, user_indices, item_indices):
        """Predict ratings for given user-item pairs using bias model"""
        predictions = np.zeros(len(user_indices))
        
        for i, (u, m) in enumerate(zip(user_indices, item_indices)):
            predictions[i] = self.global_avg + self.user_biases[u] + self.item_biases[m]
            
        # Clip predictions to valid rating range [0.5, 5.0]
        return np.clip(predictions, 0.5, 5.0)
    
    def predict_matrix(self):
        """Generate a full prediction matrix using the bias model"""
        n_users, n_items = len(self.user_biases), len(self.item_biases)
        
        # Create meshgrid for broadcasting
        users_grid, items_grid = np.meshgrid(np.arange(n_users), np.arange(n_items), indexing='ij')
        
        # Calculate predictions using broadcasting
        predictions = self.global_avg + self.user_biases[users_grid] + self.item_biases[items_grid]
        
        # Clip predictions to valid rating range
        return np.clip(predictions, 0.5, 5.0)

# --------- 3. SIMILARITY-BASED COLLABORATIVE FILTERING ---------

class ItemBasedCF:
    """Item-based collaborative filtering using cosine similarity"""
    
    def __init__(self, k=50):
        self.k = k  # Number of similar items to consider
        self.train_matrix = None
        self.item_similarity = None
    
    def fit(self, train_matrix, calculate_similarity=True):
        """
        Calculate item-item similarity matrix
        
        Args:
            train_matrix: User-item rating matrix
            calculate_similarity: Whether to calculate similarity matrix or use pre-computed one
        """
        start_time = time.time()
        self.train_matrix = train_matrix
        
        if calculate_similarity:
            print("Calculating item-item similarity matrix...")
            # Normalize the ratings by subtracting user means
            user_means = np.array([rating.mean() if rating.size > 0 else 0 
                                  for rating in [train_matrix[u].data for u in range(train_matrix.shape[0])]])
            
            # Create a normalized matrix for better similarity calculation
            normalized_matrix = train_matrix.copy()
            for u in range(train_matrix.shape[0]):
                u_ratings = normalized_matrix[u].nonzero()[1]
                if len(u_ratings) > 0:
                    normalized_matrix[u, u_ratings] = train_matrix[u, u_ratings].toarray()[0] - user_means[u]
            
            # Calculate item similarities using cosine similarity
            self.item_similarity = cosine_similarity(normalized_matrix.T)
            
            # Zero out self-similarity to avoid trivial recommendations
            np.fill_diagonal(self.item_similarity, 0)
            
            print(f"Item similarity matrix computed. Shape: {self.item_similarity.shape}")
            print(f"Training time: {time.time() - start_time:.2f} seconds")
        else:
            print("Using pre-computed similarity matrix")
        
        return self
    
    def predict(self, user_idx, item_idx):
        """
        Predict rating for a user-item pair
        
        Args:
            user_idx: User index
            item_idx: Item index
            
        Returns:
            Predicted rating
        """
        # Get items rated by the user
        user_ratings = self.train_matrix[user_idx].toarray().flatten()
        rated_items = np.where(user_ratings > 0)[0]
        
        if len(rated_items) == 0:
            # If user has no ratings, return global average
            return np.mean(self.train_matrix.data)
        
        # Get similarity scores between the target item and all items rated by the user
        similarities = self.item_similarity[item_idx, rated_items]
        
        # Find top k similar items
        if len(similarities) > self.k:
            top_indices = np.argsort(similarities)[-self.k:]
            similarities = similarities[top_indices]
            rated_items = rated_items[top_indices]
        
        # If no similar items, return user's average rating
        if len(similarities) == 0 or np.sum(np.abs(similarities)) == 0:
            return np.mean(user_ratings[rated_items]) if len(rated_items) > 0 else np.mean(self.train_matrix.data)
        
        # Calculate weighted average of ratings from similar items
        weights = similarities
        ratings = user_ratings[rated_items]
        
        # Ensure we don't divide by zero
        sum_weights = np.sum(np.abs(weights))
        if sum_weights > 0:
            prediction = np.sum(weights * ratings) / sum_weights
        else:
            prediction = np.mean(ratings)
        
        # Clip to valid rating range
        return np.clip(prediction, 0.5, 5.0)
    
    def predict_for_user(self, user_idx, item_indices=None):
        """
        Generate predictions for a user on specified items
        
        Args:
            user_idx: User index
            item_indices: List of item indices to predict ratings for (default: all items)
            
        Returns:
            Array of predicted ratings
        """
        if item_indices is None:
            item_indices = np.arange(self.train_matrix.shape[1])
        
        predictions = np.array([self.predict(user_idx, item_idx) for item_idx in item_indices])
        return predictions

class UserBasedCF:
    """User-based collaborative filtering using cosine similarity"""
    
    def __init__(self, k=50):
        self.k = k  # Number of similar users to consider
        self.train_matrix = None
        self.user_similarity = None
    
    def fit(self, train_matrix, calculate_similarity=True):
        """
        Calculate user-user similarity matrix
        
        Args:
            train_matrix: User-item rating matrix
            calculate_similarity: Whether to calculate similarity matrix or use pre-computed one
        """
        start_time = time.time()
        self.train_matrix = train_matrix
        
        if calculate_similarity:
            print("Calculating user-user similarity matrix...")
            # Computing user-user similarity is expensive for large datasets
            # For better efficiency, we'll normalize the data first
            
            # Normalize ratings by user means
            user_means = np.array([rating.mean() if rating.size > 0 else 0 
                                  for rating in [train_matrix[u].data for u in range(train_matrix.shape[0])]])
            
            # Create a normalized matrix for better similarity calculation
            normalized_matrix = train_matrix.copy()
            for u in range(train_matrix.shape[0]):
                u_ratings = normalized_matrix[u].nonzero()[1]
                if len(u_ratings) > 0:
                    normalized_matrix[u, u_ratings] = train_matrix[u, u_ratings].toarray()[0] - user_means[u]
            
            # Calculate user similarities using cosine similarity
            # This is memory-intensive for large datasets
            self.user_similarity = cosine_similarity(normalized_matrix)
            
            # Zero out self-similarity to avoid trivial recommendations
            np.fill_diagonal(self.user_similarity, 0)
            
            print(f"User similarity matrix computed. Shape: {self.user_similarity.shape}")
            print(f"Training time: {time.time() - start_time:.2f} seconds")
        else:
            print("Using pre-computed similarity matrix")
        
        return self
    
    def predict(self, user_idx, item_idx):
        """
        Predict rating for a user-item pair
        
        Args:
            user_idx: User index
            item_idx: Item index
            
        Returns:
            Predicted rating
        """
        # Get users who rated the item
        item_ratings = self.train_matrix[:, item_idx].toarray().flatten()
        users_rated = np.where(item_ratings > 0)[0]
        
        if len(users_rated) == 0:
            # If no user rated this item, return the user's average rating
            user_ratings = self.train_matrix[user_idx].data
            return np.mean(user_ratings) if len(user_ratings) > 0 else np.mean(self.train_matrix.data)
        
        # Get similarity scores between current user and users who rated the item
        similarities = self.user_similarity[user_idx, users_rated]
        
        # Find top k similar users
        if len(similarities) > self.k:
            top_indices = np.argsort(similarities)[-self.k:]
            similarities = similarities[top_indices]
            users_rated = users_rated[top_indices]
        
        # If no similar users, return global average
        if len(similarities) == 0 or np.sum(np.abs(similarities)) == 0:
            return np.mean(self.train_matrix.data)
        
        # Calculate weighted average of ratings from similar users
        weights = similarities
        ratings = item_ratings[users_rated]
        
        # Ensure we don't divide by zero
        sum_weights = np.sum(np.abs(weights))
        if sum_weights > 0:
            prediction = np.sum(weights * ratings) / sum_weights
        else:
            prediction = np.mean(ratings)
        
        # Clip to valid rating range
        return np.clip(prediction, 0.5, 5.0)
    
    def predict_for_user(self, user_idx, item_indices=None):
        """
        Generate predictions for a user on specified items
        
        Args:
            user_idx: User index
            item_indices: List of item indices to predict ratings for (default: all items)
            
        Returns:
            Array of predicted ratings
        """
        if item_indices is None:
            item_indices = np.arange(self.train_matrix.shape[1])
        
        predictions = np.array([self.predict(user_idx, item_idx) for item_idx in item_indices])
        return predictions

# --------- 4. MATRIX FACTORIZATION MODELS ---------

class SVDModel:
    """Singular Value Decomposition model for collaborative filtering"""
    
    def __init__(self, n_factors=100, regularization=0.1, n_epochs=20, learning_rate=0.005):
        self.n_factors = n_factors
        self.regularization = regularization
        self.n_epochs = n_epochs
        self.learning_rate = learning_rate
        self.global_avg = None
        self.user_biases = None
        self.item_biases = None
        self.user_factors = None
        self.item_factors = None
    
    def fit(self, train_matrix, val_matrix=None):
        """
        Train the SVD model using Stochastic Gradient Descent
        
        Args:
            train_matrix: Training user-item rating matrix
            val_matrix: Validation user-item rating matrix for early stopping
        """
        start_time = time.time()
        n_users, n_items = train_matrix.shape
        
        # Initialize model parameters
        self.global_avg = np.mean(train_matrix.data)
        self.user_biases = np.zeros(n_users)
        self.item_biases = np.zeros(n_items)
        
        # Initialize latent factors
        self.user_factors = np.random.normal(0, 0.1, (n_users, self.n_factors))
        self.item_factors = np.random.normal(0, 0.1, (n_items, self.n_factors))
        
        # Prepare training data
        users, items = train_matrix.nonzero()
        ratings = np.array(train_matrix[users, items]).flatten()
        
        # Prepare validation data if provided
        if val_matrix is not None:
            val_users, val_items = val_matrix.nonzero()
            val_ratings = np.array(val_matrix[val_users, val_items]).flatten()
        
        best_val_rmse = float('inf')
        best_epoch = 0
        
        print("Training SVD model with SGD...")
        for epoch in range(self.n_epochs):
            epoch_start = time.time()
            
            # Shuffle training data
            indices = np.arange(len(users))
            np.random.shuffle(indices)
            
            # SGD updates
            for idx in indices:
                u, i, r = users[idx], items[idx], ratings[idx]
                
                # Compute predicted rating
                pred = self.global_avg + self.user_biases[u] + self.item_biases[i] + np.dot(self.user_factors[u], self.item_factors[i])
                
                # Compute error
                error = r - pred
                
                # Update biases
                self.user_biases[u] += self.learning_rate * (error - self.regularization * self.user_biases[u])
                self.item_biases[i] += self.learning_rate * (error - self.regularization * self.item_biases[i])
                
                # Update latent factors
                user_factor = self.user_factors[u].copy()
                item_factor = self.item_factors[i].copy()
                
                self.user_factors[u] += self.learning_rate * (error * item_factor - self.regularization * user_factor)
                self.item_factors[i] += self.learning_rate * (error * user_factor - self.regularization * item_factor)
            
            # Compute training RMSE
            train_preds = np.array([self.global_avg + self.user_biases[u] + self.item_biases[i] + 
                                  np.dot(self.user_factors[u], self.item_factors[i]) 
                                  for u, i in zip(users, items)])
            train_rmse = np.sqrt(mean_squared_error(ratings, train_preds))
            
            # Compute validation RMSE if validation data is provided
            if val_matrix is not None:
                val_preds = np.array([self.global_avg + self.user_biases[u] + self.item_biases[i] + 
                                    np.dot(self.user_factors[u], self.item_factors[i]) 
                                    for u, i in zip(val_users, val_items)])
                val_rmse = np.sqrt(mean_squared_error(val_ratings, val_preds))
                
                # Early stopping check
                if val_rmse < best_val_rmse:
                    best_val_rmse = val_rmse
                    best_epoch = epoch
                elif epoch - best_epoch >= 3:  # Stop if no improvement for 3 epochs
                    print(f"Early stopping at epoch {epoch}")
                    break
                
                print(f"Epoch {epoch+1}/{self.n_epochs} - "
                      f"Train RMSE: {train_rmse:.4f} - "
                      f"Val RMSE: {val_rmse:.4f} - "
                      f"Time: {time.time() - epoch_start:.2f}s")
            else:
                print(f"Epoch {epoch+1}/{self.n_epochs} - "
                      f"Train RMSE: {train_rmse:.4f} - "
                      f"Time: {time.time() - epoch_start:.2f}s")
        
        print(f"Total training time: {time.time() - start_time:.2f} seconds")
        return self
    
    def predict(self, user_indices, item_indices):
        """
        Predict ratings for given user-item pairs
        
        Args:
            user_indices: List of user indices
            item_indices: List of item indices
            
        Returns:
            Array of predicted ratings
        """
        predictions = np.zeros(len(user_indices))
        
        for i, (u, m) in enumerate(zip(user_indices, item_indices)):
            # Compute prediction using the learned model
            pred = self.global_avg + self.user_biases[u] + self.item_biases[m] + np.dot(self.user_factors[u], self.item_factors[m])
            # Clip to valid rating range
            predictions[i] = np.clip(pred, 0.5, 5.0)
            
        return predictions
    
    def predict_all(self):
        """Generate full prediction matrix using the trained model"""
        n_users, n_items = len(self.user_biases), len(self.item_biases)
        predictions = np.zeros((n_users, n_items))
        
        # Compute dot product between all user and item factors
        dot_products = np.dot(self.user_factors, self.item_factors.T)
        
        # Add biases using broadcasting
        predictions = (self.global_avg + 
                      self.user_biases.reshape(-1, 1) + 
                      self.item_biases.reshape(1, -1) + 
                      dot_products)
        
        # Clip to valid rating range
        return np.clip(predictions, 0.5, 5.0)

class SVDppModel:
    """
    SVD++ model incorporating implicit feedback
    This is a simplified implementation of the full SVD++ algorithm
    """
    
    def __init__(self, n_factors=100, regularization=0.1, n_epochs=20, learning_rate=0.005):
        self.n_factors = n_factors
        self.regularization = regularization
        self.n_epochs = n_epochs
        self.learning_rate = learning_rate
        self.global_avg = None
        self.user_biases = None
        self.item_biases = None
        self.user_factors = None
        self.item_factors = None
        self.implicit_factors = None  # Implicit feedback factors
    
    def fit(self, train_matrix, val_matrix=None):
        """
        Train the SVD++ model using Stochastic Gradient Descent
        
        Args:
            train_matrix: Training user-item rating matrix
            val_matrix: Validation user-item rating matrix for early stopping
        """
        start_time = time.time()
        n_users, n_items = train_matrix.shape
        
        # Initialize model parameters
        self.global_avg = np.mean(train_matrix.data)
        self.user_biases = np.zeros(n_users)
        self.item_biases = np.zeros(n_items)
        
        # Initialize latent factors
        self.user_factors = np.random.normal(0, 0.1, (n_users, self.n_factors))
        self.item_factors = np.random.normal(0, 0.1, (n_items, self.n_factors))
        self.implicit_factors = np.random.normal(0, 0.1, (n_items, self.n_factors))
        
        # Prepare training data
        users, items = train_matrix.nonzero()
        ratings = np.array(train_matrix[users, items]).flatten()
        
        # Create user to items mapping for implicit feedback
        user_to_items = {}
        for u, i in zip(users, items):
            if u not in user_to_items:
                user_to_items[u] = []
            user_to_items[u].append(i)
        
        # Prepare validation data if provided
        if val_matrix is not None:
            val_users, val_items = val_matrix.nonzero()
            val_ratings = np.array(val_matrix[val_users, val_items]).flatten()
        
        best_val_rmse = float('inf')
        best_epoch = 0
        
        print("Training SVD++ model with SGD...")
        for epoch in range(self.n_epochs):
            epoch_start = time.time()
            
            # Shuffle training data
            indices = np.arange(len(users))
            np.random.shuffle(indices)
            
            # SGD updates
            for idx in indices:
                u, i, r = users[idx], items[idx], ratings[idx]
                
                # Get items rated by user u (implicit feedback)
                rated_items = user_to_items.get(u, [])
                implicit_sum = np.zeros(self.n_factors)
                
                if rated_items:
                    # Compute the sum of implicit factors
                    for j in rated_items:
                        implicit_sum += self.implicit_factors[j]
                    
                    # Normalize by the square root of the number of items
                    implicit_sum /= np.sqrt(len(rated_items))
                
                # Compute predicted rating (SVD++ formula)
                pred = (self.global_avg + 
                        self.user_biases[u] + 
                        self.item_biases[i] + 
                        np.dot(self.item_factors[i], self.user_factors[u] + implicit_sum))
                
                # Compute error
                error = r - pred
                
                # Update biases
                self.user_biases[u] += self.learning_rate * (error - self.regularization * self.user_biases[u])
                self.item_biases[i] += self.learning_rate * (error - self.regularization * self.item_biases[i])
                
                # Update latent factors
                user_factor = self.user_factors[u].copy()
                item_factor = self.item_factors[i].copy()
                
                # Update user factors
                self.user_factors[u] += self.learning_rate * (error * item_factor - self.regularization * user_factor)
                
                # Update item factors
                self.item_factors[i] += self.learning_rate * (error * (user_factor + implicit_sum) - self.regularization * item_factor)
                
                # Update implicit feedback factors
                if rated_items:
                    factor_update = self.learning_rate * (error * item_factor / np.sqrt(len(rated_items)) - self.regularization * self.implicit_factors)
                    for j in rated_items:
                        self.implicit_factors[j] += factor_update[j]
            
            # Compute training RMSE (simplified for efficiency)
            sample_size = min(10000, len(users))
            sample_indices = np.random.choice(len(users), sample_size, replace=False)
            
            sample_preds = []
            sample_true = []
            
            for idx in sample_indices:
                u, i, r = users[idx], items[idx], ratings[idx]
                rated_items = user_to_items.get(u, [])
                implicit_sum = np.zeros(self.n_factors)
                
                if rated_items:
                    for j in rated_items:
                        implicit_sum += self.implicit_factors[j]
                    implicit_sum /= np.sqrt(len(rated_items))
                
                pred = (self.global_avg + 
                        self.user_biases[u] + 
                        self.item_biases[i] + 
                        np.dot(self.item_factors[i], self.user_factors[u] + implicit_sum))
                
                sample_preds.append(np.clip(pred, 0.5, 5.0))
                sample_true.append(r)
            
            train_rmse = np.sqrt(mean_squared_error(sample_true, sample_preds))
            
            # Compute validation RMSE if validation data is provided
            if val_matrix is not None:
                val_sample_size = min(10000, len(val_users))
                val_sample_indices = np.random.choice(len(val_users), val_sample_size, replace=False)
                
                val_sample_preds = []
                val_sample_true = []
                
                for idx in val_sample_indices:
                    u, i, r = val_users[idx], val_items[idx], val_ratings[idx]
                    rated_items = user_to_items.get(u, [])
                    implicit_sum = np.zeros(self.n_factors)
                    
                    if rated_items:
                        for j in rated_items:
                            implicit_sum += self.implicit_factors[j]
                        implicit_sum /= np.sqrt(len(rated_items))
                    
                    pred = (self.global_avg + 
                            self.user_biases[u] + 
                            self.item_biases[i] + 
                            np.dot(self.item_factors[i], self.user_factors[u] + implicit_sum))
                    
                    val_sample_preds.append(np.clip(pred, 0.5, 5.0))
                    val_sample_true.append(r)
                
                val_rmse = np.sqrt(mean_squared_error(val_sample_true, val_sample_preds))
                
                # Early stopping check
                if val_rmse < best_val_rmse:
                    best_val_rmse = val_rmse
                    best_epoch = epoch
                elif epoch - best_epoch >= 3:  # Stop if no improvement for 3 epochs
                    print(f"Early stopping at epoch {epoch}")
                    break
                
                print(f"Epoch {epoch+1}/{self.n_epochs} - "
                      f"Train RMSE: {train_rmse:.4f} - "
                      f"Val RMSE: {val_rmse:.4f} - "
                      f"Time: {time.time() - epoch_start:.2f}s")
            else:
                print(f"Epoch {epoch+1}/{self.n_epochs} - "
                      f"Train RMSE: {train_rmse:.4f} - "
                      f"Time: {time.time() - epoch_start:.2f}s")
        
        print(f"Total training time: {time.time() - start_time:.2f} seconds")
        return self
    
    def predict(self, user_idx, item_idx, user_to_items=None):
        """
        Predict rating for a single user-item pair
        
        Args:
            user_idx: User index
            item_idx: Item index
            user_to_items: Dictionary mapping user indices to list of rated item indices
            
        Returns:
            Predicted rating
        """
        # Initialize implicit sum
        implicit_sum = np.zeros(self.n_factors)
        
        # Compute implicit feedback component if rated items are provided
        if user_to_items is not None and user_idx in user_to_items:
            rated_items = user_to_items[user_idx]
            if rated_items:
                for j in rated_items:
                    implicit_sum += self.implicit_factors[j]
                implicit_sum /= np.sqrt(len(rated_items))
        
        # Compute prediction using SVD++ formula
        pred = (self.global_avg + 
                self.user_biases[user_idx] + 
                self.item_biases[item_idx] + 
                np.dot(self.item_factors[item_idx], self.user_factors[user_idx] + implicit_sum))
        
        # Clip to valid rating range
        return np.clip(pred, 0.5, 5.0)
    
    def predict_for_user(self, user_idx, item_indices=None, train_matrix=None):
        """
        Generate predictions for a user on specified items
        
        Args:
            user_idx: User index
            item_indices: List of item indices to predict ratings for (default: all items)
            train_matrix: Training matrix for extracting implicit feedback information
            
        Returns:
            Array of predicted ratings
        """
        if item_indices is None:
            item_indices = np.arange(len(self.item_biases))
        
        # Extract rated items for the user from train_matrix
        user_to_items = {}
        if train_matrix is not None:
            rated_items = train_matrix[user_idx].nonzero()[1]
            user_to_items[user_idx] = rated_items
        
        predictions = np.zeros(len(item_indices))
        
        # Compute implicit sum once for efficiency
        implicit_sum = np.zeros(self.n_factors)
        if user_to_items and user_idx in user_to_items:
            rated_items = user_to_items[user_idx]
            if len(rated_items) > 0:
                for j in rated_items:
                    implicit_sum += self.implicit_factors[j]
                implicit_sum /= np.sqrt(len(rated_items))
        
        # Generate predictions for each item
        for i, item_idx in enumerate(item_indices):
            # Compute prediction using SVD++ formula
            pred = (self.global_avg + 
                    self.user_biases[user_idx] + 
                    self.item_biases[item_idx] + 
                    np.dot(self.item_factors[item_idx], self.user_factors[user_idx] + implicit_sum))
            
            # Clip to valid rating range
            predictions[i] = np.clip(pred, 0.5, 5.0)
        
        return predictions

# --------- 5. HYBRID RECOMMENDATION MODEL ---------

class HybridRecommender:
    """
    Hybrid recommendation model that combines multiple recommendation models
    """
    
    def __init__(self, models, weights=None):
        """
        Initialize the hybrid recommender
        
        Args:
            models: List of recommendation models
            weights: List of weights for each model (default: equal weights)
        """
        self.models = models
        self.weights = weights if weights is not None else [1.0/len(models)] * len(models)
        
        # Normalize weights to sum to 1
        self.weights = np.array(self.weights) / sum(self.weights)
    
    def predict(self, user_idx, item_idx, **kwargs):
        """
        Generate a prediction by combining predictions from multiple models
        
        Args:
            user_idx: User index
            item_idx: Item index
            **kwargs: Additional arguments to pass to the models
            
        Returns:
            Weighted prediction
        """
        predictions = []
        
        for model in self.models:
            if hasattr(model, 'predict'):
                if 'user_idx' in kwargs and 'item_idx' in kwargs:
                    pred = model.predict(kwargs['user_idx'], kwargs['item_idx'])
                else:
                    pred = model.predict(user_idx, item_idx)
                predictions.append(pred)
        
        # Weighted average of predictions
        if predictions:
            return np.average(predictions, weights=self.weights[:len(predictions)])
        else:
            return None
    
    def predict_for_user(self, user_idx, item_indices=None, **kwargs):
        """
        Generate predictions for a user on multiple items
        
        Args:
            user_idx: User index
            item_indices: List of item indices to predict ratings for
            **kwargs: Additional arguments to pass to the models
            
        Returns:
            Array of weighted predictions
        """
        if item_indices is None:
            # Assume all models have the same number of items
            for model in self.models:
                if hasattr(model, 'item_biases'):
                    item_indices = np.arange(len(model.item_biases))
                    break
        
        all_predictions = []
        
        for model in self.models:
            if hasattr(model, 'predict_for_user'):
                preds = model.predict_for_user(user_idx, item_indices, **kwargs)
                all_predictions.append(preds)
        
        # Weighted average of predictions
        if all_predictions:
            weighted_preds = np.zeros(len(item_indices))
            for i, preds in enumerate(all_predictions):
                weighted_preds += self.weights[i] * preds
            return weighted_preds
        else:
            return None
    
    def recommend_items(self, user_idx, n=10, exclude_rated=True, train_matrix=None, **kwargs):
        """
        Recommend top N items for a user
        
        Args:
            user_idx: User index
            n: Number of recommendations
            exclude_rated: Whether to exclude already rated items
            train_matrix: Training matrix for excluding rated items
            **kwargs: Additional arguments to pass to the models
            
        Returns:
            List of top N recommended item indices
        """
        # Get predictions for all items
        predictions = self.predict_for_user(user_idx, **kwargs)
        
        if exclude_rated and train_matrix is not None:
            # Get items the user has already rated
            rated_items = train_matrix[user_idx].nonzero()[1]
            # Set predictions for rated items to -inf to exclude them
            predictions[rated_items] = -np.inf
        
        # Get top N items
        top_items = np.argsort(predictions)[::-1][:n]
        
        return top_items

# --------- 6. RECOMMENDATION SYSTEM MANAGER ---------

class MovieRecommendationSystem:
    """
    Main recommendation system manager that handles model training, evaluation, and recommendation generation
    """
    
    def __init__(self, data_dir="processed_data"):
        """
        Initialize the recommendation system
        
        Args:
            data_dir: Directory containing processed data files
        """
        # Load data
        self.data = load_processed_data(data_dir)
        
        # Initialize models
        self.models = {}
        self.best_model = None
        self.evaluation_results = []
    
    def train_models(self, models_to_train=None):
        """
        Train multiple recommendation models
        
        Args:
            models_to_train: List of model names to train (default: all models)
        """
        available_models = {
            'global_avg': self._train_global_avg,
            'user_item_bias': self._train_user_item_bias,
            'item_cf': self._train_item_cf,
            'user_cf': self._train_user_cf,
            'svd': self._train_svd,
            'svdpp': self._train_svdpp,
            'hybrid': self._train_hybrid
        }
        
        if models_to_train is None:
            models_to_train = list(available_models.keys())
        
        for model_name in models_to_train:
            if model_name in available_models:
                print(f"\n=== Training {model_name} model ===")
                self.models[model_name] = available_models[model_name]()
            else:
                print(f"Unknown model: {model_name}")
    
    def _train_global_avg(self):
        """Train global average baseline model"""
        model = GlobalAverageModel()
        return model.fit(self.data['train_matrix'])
    
    def _train_user_item_bias(self):
        """Train user-item bias baseline model"""
        model = UserItemBiasModel()
        return model.fit(self.data['train_matrix'])
    
    def _train_item_cf(self):
        """Train item-based collaborative filtering model"""
        model = ItemBasedCF(k=50)
        return model.fit(self.data['train_matrix'])
    
    def _train_user_cf(self):
        """Train user-based collaborative filtering model"""
        model = UserBasedCF(k=50)
        return model.fit(self.data['train_matrix'])
    
    def _train_svd(self):
        """Train SVD model"""
        model = SVDModel(n_factors=50, regularization=0.1, n_epochs=20, learning_rate=0.005)
        return model.fit(self.data['train_matrix'], self.data['val_matrix'])
    
    def _train_svdpp(self):
        """Train SVD++ model"""
        model = SVDppModel(n_factors=50, regularization=0.1, n_epochs=20, learning_rate=0.005)
        return model.fit(self.data['train_matrix'], self.data['val_matrix'])
    
    def _train_hybrid(self):
        """Train hybrid model combining multiple models"""
        models = []
        weights = []
        
        # Add trained models to the hybrid
        if 'user_item_bias' in self.models:
            models.append(self.models['user_item_bias'])
            weights.append(0.1)
        
        if 'item_cf' in self.models:
            models.append(self.models['item_cf'])
            weights.append(0.3)
        
        if 'svd' in self.models:
            models.append(self.models['svd'])
            weights.append(0.6)
        
        if not models:
            print("No component models available for hybrid. Training defaults...")
            models = [self._train_user_item_bias(), self._train_item_cf(), self._train_svd()]
            weights = [0.1, 0.3, 0.6]
        
        print(f"Creating hybrid model with {len(models)} component models")
        return HybridRecommender(models, weights)
    
    def evaluate_models(self, test_sample_size=10000):
        """
        Evaluate all trained models on test data
        
        Args:
            test_sample_size: Number of test ratings to sample (for efficiency)
        
        Returns:
            List of evaluation results for each model
        """
        print("\n=== Evaluating Models ===")
        
        # Sample test data
        test = self.data['test']
        if len(test) > test_sample_size:
            test_sample = test.sample(n=test_sample_size, random_state=42)
        else:
            test_sample = test
        
        user_indices = [self.data['user_map'][uid] for uid in test_sample['userId']]
        item_indices = [self.data['movie_map'][mid] for mid in test_sample['movieId']]
        true_ratings = test_sample['rating'].values
        
        self.evaluation_results = []
        
        for model_name, model in self.models.items():
            print(f"\nEvaluating {model_name} model...")
            
            # Generate predictions
            if model_name == 'user_cf' or model_name == 'item_cf':
                # These models don't support batch prediction
                predictions = []
                for u, i in zip(user_indices, item_indices):
                    try:
                        pred = model.predict(u, i)
                        predictions.append(pred)
                    except Exception as e:
                        print(f"Error predicting for user {u}, item {i}: {e}")
                        predictions.append(3.0)  # Default prediction
                predictions = np.array(predictions)
            else:
                predictions = model.predict(user_indices, item_indices)
            
            # Evaluate
            result = evaluate_model(predictions, true_ratings, name=model_name)
            self.evaluation_results.append(result)
        
        # Find the best model
        if self.evaluation_results:
            # Find model with lowest RMSE
            best_idx = np.argmin([r['rmse'] for r in self.evaluation_results])
            best_model_name = self.evaluation_results[best_idx]['model']
            self.best_model = self.models[best_model_name]
            
            print(f"\nBest performing model: {best_model_name} with RMSE: {self.evaluation_results[best_idx]['rmse']:.4f}")
        
        return self.evaluation_results
    
    def recommend_for_user(self, user_id, n=10, model_name=None, exclude_rated=True):
        """
        Generate movie recommendations for a user
        
        Args:
            user_id: User ID (raw ID, not matrix index)
            n: Number of recommendations
            model_name: Name of model to use (default: best model)
            exclude_rated: Whether to exclude already rated movies
        
        Returns:
            DataFrame with recommended movies and their details
        """
        if model_name is None and self.best_model is not None:
            model = self.best_model
            model_name = next(name for name, m in self.models.items() if m is self.best_model)
        elif model_name in self.models:
            model = self.models[model_name]
        else:
            raise ValueError(f"Model '{model_name}' not found or no best model available")
        
        # Convert user ID to matrix index
        if user_id in self.data['user_map']:
            user_idx = self.data['user_map'][user_id]
        else:
            raise ValueError(f"User ID {user_id} not found in the dataset")
        
        print(f"Generating recommendations for user {user_id} using {model_name} model...")
        
        # Get recommendations
        if hasattr(model, 'recommend_items'):
            # For hybrid model
            top_items = model.recommend_items(
                user_idx, n=n, exclude_rated=exclude_rated, 
                train_matrix=self.data['train_matrix']
            )
        elif model_name in ['user_cf', 'item_cf']:
            # For CF models
            predictions = model.predict_for_user(user_idx)
            if exclude_rated:
                # Exclude already rated items
                rated_items = self.data['train_matrix'][user_idx].nonzero()[1]
                predictions[rated_items] = -np.inf
            top_items = np.argsort(predictions)[::-1][:n]
        else:
            # For other models
            if hasattr(model, 'predict_all'):
                # For models that can generate all predictions at once
                predictions = model.predict_all()[user_idx]
            else:
                # For models that predict one at a time
                all_items = np.arange(self.data['train_matrix'].shape[1])
                predictions = np.array([model.predict(user_idx, i) for i in all_items])
                
            if exclude_rated:
                # Exclude already rated items
                rated_items = self.data['train_matrix'][user_idx].nonzero()[1]
                predictions[rated_items] = -np.inf
            
            top_items = np.argsort(predictions)[::-1][:n]
        
        # Convert matrix indices back to movie IDs
        movie_ids = [self.data['movie_map_rev'][idx] for idx in top_items]
        
        # Get movie details
        movies_df = self.data['movies']
        recommended_movies = movies_df[movies_df['movieId'].isin(movie_ids)].copy()
        
        # Add predicted ratings
        if hasattr(model, 'predict'):
            predicted_ratings = [model.predict(user_idx, item_idx) for item_idx in top_items]
            movie_id_to_rating = dict(zip(movie_ids, predicted_ratings))
            recommended_movies['predicted_rating'] = recommended_movies['movieId'].map(movie_id_to_rating)
        
        # Sort by predicted rating
        if 'predicted_rating' in recommended_movies.columns:
            recommended_movies = recommended_movies.sort_values('predicted_rating', ascending=False)
        
        return recommended_movies[['movieId', 'title', 'genres', 'predicted_rating']].reset_index(drop=True)
    
    def get_similar_movies(self, movie_id, n=10, model_name='item_cf'):
        """
        Find similar movies to a given movie
        
        Args:
            movie_id: Movie ID (raw ID, not matrix index)
            n: Number of similar movies to return
            model_name: Name of model to use (only 'item_cf' and 'svd' supported)
        
        Returns:
            DataFrame with similar movies and their details
        """
        if model_name not in ['item_cf', 'svd']:
            raise ValueError("Only 'item_cf' and 'svd' models support similar movie recommendations")
        
        if model_name not in self.models:
            raise ValueError(f"Model '{model_name}' not found")
        
        model = self.models[model_name]
        
        # Convert movie ID to matrix index
        if movie_id in self.data['movie_map']:
            movie_idx = self.data['movie_map'][movie_id]
        else:
            raise ValueError(f"Movie ID {movie_id} not found in the dataset")
        
        print(f"Finding movies similar to movie {movie_id} using {model_name} model...")
        
        # Get similar movies
        if model_name == 'item_cf':
            # Get similarity scores
            similarity_scores = model.item_similarity[movie_idx]
            # Get top similar movies
            similar_indices = np.argsort(similarity_scores)[::-1][1:n+1]  # Skip the first one (self)
            similarities = similarity_scores[similar_indices]
        elif model_name == 'svd':
            # Use item factors to compute similarity
            target_factors = model.item_factors[movie_idx]
            # Compute cosine similarity with all other items
            similarities = cosine_similarity([target_factors], model.item_factors)[0]
            # Get top similar movies
            similar_indices = np.argsort(similarities)[::-1][1:n+1]  # Skip the first one (self)
            similarities = similarities[similar_indices]
        
        # Convert matrix indices back to movie IDs
        movie_ids = [self.data['movie_map_rev'][idx] for idx in similar_indices]
        
        # Get movie details
        movies_df = self.data['movies']
        similar_movies = movies_df[movies_df['movieId'].isin(movie_ids)].copy()
        
        # Add similarity scores
        movie_id_to_similarity = dict(zip(movie_ids, similarities))
        similar_movies['similarity_score'] = similar_movies['movieId'].map(movie_id_to_similarity)
        
        # Sort by similarity
        similar_movies = similar_movies.sort_values('similarity_score', ascending=False)
        
        return similar_movies[['movieId', 'title', 'genres', 'similarity_score']].reset_index(drop=True)
    
    def plot_evaluation_results(self):
        """Plot evaluation results for all models"""
        if not self.evaluation_results:
            print("No evaluation results to plot. Run evaluate_models() first.")
            return
        
        # Create dataframe from evaluation results
        results_df = pd.DataFrame(self.evaluation_results)
        
        # Plot RMSE
        plt.figure(figsize=(12, 6))
        barplot = sns.barplot(x='model', y='rmse', data=results_df)
        plt.title('RMSE by Model', fontsize=16)
        plt.xlabel('Model', fontsize=14)
        plt.ylabel('RMSE (lower is better)', fontsize=14)
        plt.xticks(rotation=45)
        
        # Add value labels
        for patch in barplot.patches:
            barplot.annotate(f"{patch.get_height():.4f}",
                          (patch.get_x() + patch.get_width() / 2, patch.get_height()),
                          ha='center', va='bottom', fontsize=10)
        
        plt.tight_layout()
        plt.show()
        
        # Plot accuracy within thresholds
        plt.figure(figsize=(12, 6))
        results_melted = pd.melt(results_df, id_vars=['model'], value_vars=['within_05', 'within_1'],
                              var_name='threshold', value_name='accuracy')
        results_melted['threshold'] = results_melted['threshold'].map({
            'within_05': 'Within 0.5 stars',
            'within_1': 'Within 1.0 stars'
        })
        
        barplot = sns.barplot(x='model', y='accuracy', hue='threshold', data=results_melted)
        plt.title('Prediction Accuracy by Model', fontsize=16)
        plt.xlabel('Model', fontsize=14)
        plt.ylabel('Accuracy (higher is better)', fontsize=14)
        plt.xticks(rotation=45)
        plt.legend(title='Threshold')
        
        # Add value labels
        for patch in barplot.patches:
            barplot.annotate(f"{patch.get_height():.2%}",
                          (patch.get_x() + patch.get_width() / 2, patch.get_height()),
                          ha='center', va='bottom', fontsize=8)
        
        plt.tight_layout()
        plt.show()

# Main function to run the recommendation system
def main():
    # Create recommendation system
    rec_system = MovieRecommendationSystem()
    
    # Train models
    models_to_train = ['global_avg', 'user_item_bias', 'item_cf', 'svd', 'hybrid']
    rec_system.train_models(models_to_train)
    
    # Evaluate models
    rec_system.evaluate_models()
    
    # Plot evaluation results
    rec_system.plot_evaluation_results()
    
    # Generate recommendations for a sample user
    user_id = 1  # Replace with a valid user ID
    recommendations = rec_system.recommend_for_user(user_id, n=10)
    print("\nTop 10 movie recommendations for user:", user_id)
    print(recommendations)
    
    # Find similar movies to a sample movie
    movie_id = 1  # Replace with a valid movie ID
    similar_movies = rec_system.get_similar_movies(movie_id, n=10)
    print("\nTop 10 movies similar to movie:", movie_id)
    print(similar_movies)

if __name__ == "__main__":
    main()

Loading processed data...
Data loading complete!

=== Training global_avg model ===
Global average rating: 3.5417
Training time: 0.05 seconds

=== Training user_item_bias model ===
Global average: 3.5417
User bias range: [-3.0357, 1.4582]
Item bias range: [-3.0042, 1.3888]
Training time: 1583.19 seconds

=== Training item_cf model ===
Calculating item-item similarity matrix...
